In [2]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from imblearn.under_sampling import InstanceHardnessThreshold, RandomUnderSampler, EditedNearestNeighbours
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import CLFS, CV, SCORERS
from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob,FilteringEstimator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
class Spyder(BaseEstimator, ClassifierMixin):
    def __init__(self,estimator,K=5):
        self.estimator = estimator
        self.K = K
        
    def sample(self,X,Y,sample_weight):    # SPIDER2, relabel=False and ampl=weak by default
        if sample_weight is None:
            sample_weight = np.ones_like(Y)

        # First step, Remove unsafe samples from majority, 
        enn = EditedNearestNeighbours(sampling_strategy='majority',n_neighbors=5,kind_sel='mode')
        Xs,Ys = enn.fit_resample(X,Y)
        sample_weight = sample_weight[enn.sample_indices_]
            
        #Second, upsample unsafe minority samples
        disagreement = (kDN(Xs,Ys,K=5,weight='uniform') * 5).astype('int')
        agreement = 5 - disagreement

        n = disagreement - agreement  #Number of times to upsample
        n[Ys==0] = 0  #Don't upsample majority-class samples
        n[n<0] = 0

        Xt,Yt,SWt = Xs.copy(),Ys.copy(),sample_weight.copy()
        while n.sum()>0:
            Xtmp, Ytmp, SWtmp = Xs[n>0].copy(),Ys[n>0].copy(),sample_weight[n>0].copy()
            Xt,Yt,SWt = np.concatenate((Xt,Xtmp)),np.concatenate((Yt,Ytmp)),np.concatenate((SWt,SWtmp))
            
            n -= 1
            n[n==-1] = 0
            
        return Xt,Yt,SWt
    
    def fit(self, X, Y,sample_weight=None):
        Xf,Yf,sample_weight = self.sample(X, Y, sample_weight)
#         a,b = np.unique(Y,return_counts=True)[1],np.unique(Yf,return_counts=True)[1]
#         print(a.max()/a.min(),b.max()/b.min(),a,b,Xf.shape,len(Xf)/len(X))
        assert len(np.unique(Yf))==2,"Pos class completely filtered out"
        try:
            self.estimator = self.estimator.fit(Xf, Yf,sample_weight=sample_weight)
        except TypeError as e:
            self.estimator = self.estimator.fit(Xf, Yf)
        return self

    @property
    def classes_(self):
        return self.estimator.classes_

    def predict(self, X):
        return self.estimator.predict(X)
    
    def predict_proba(self, X):
        return self.estimator.predict_proba(X) 

In [5]:
models = {}
# for c,clf in CLFS.items():
#     models[('Spyder',c)] = Spyder(clf)    
    
from sklearn.ensemble import BaggingClassifier
nb = GaussianNB()
dt = DecisionTreeClassifier(max_depth=20,max_features='sqrt')
models[('Spyder',"BagNB")] = BaggingClassifier(base_estimator=Spyder(nb),n_estimators=20)
models[('Spyder',"BagDT")] = BaggingClassifier(base_estimator=Spyder(dt),n_estimators=20)
    
models.keys(),len(models)

(dict_keys([('Spyder', 'BagNB'), ('Spyder', 'BagDT')]), 2)

In [6]:
path = "Spyder_Bag.csv"
cols = pd.MultiIndex.from_product([['Spyder'],["BagNB","BagDT"],[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)

In [7]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)

0
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('Spyder', 'BagNB')
274.35 [0.284, 0.382]
('Spyder', 'BagDT')
275.95 [0.264, 0.322]

1
groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)
('Spyder', 'BagNB')
203.73 [0.25, 0.369]
('Spyder', 'BagDT')
205.26 [0.451, 0.448]

2
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)
('Spyder', 'BagNB')
243.91 [0.332, 0.46]
('Spyder', 'BagDT')
241.42 [0.3, 0.468]

3
wicket-1.3.0-incubating-beta-1.csv noise:0.164, imb:4.806,288,1384, Shape:(1672, 65)
('Spyder', 'BagNB')
221.88 [0.313, 0.442]
('Spyder', 'BagDT')
222.74 [0.324, 0.328]

4
jruby-1.1.csv noise:0.175, imb:3.540,161,570, Shape:(731, 65)
('Spyder', 'BagNB')
201.83 [0.372, 0.616]
('Spyder', 'BagDT')
203.16 [0.478, 0.595]

5
jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)
('Spyder', 'BagNB')
204.79 [0.429, 0.602]
('Spyder', 'BagDT')
206.92 [0.443, 0.581]

6
lucene-2.3.0.csv noise:0.204, imb:4.031,160,645, Shape:(805

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


264.93 [0.077, 0.166]

11
lucene-3.1.csv noise:0.120, imb:7.477,331,2475, Shape:(2806, 65)
('Spyder', 'BagNB')
271.36 [0.174, 0.21]
('Spyder', 'BagDT')
273.66 [0.174, 0.157]

12
groovy-1_6_BETA_2.csv noise:0.096, imb:7.583,103,781, Shape:(884, 65)
('Spyder', 'BagNB')
211.66 [0.232, 0.318]
('Spyder', 'BagDT')
211.23 [0.384, 0.463]

13
activemq-5.2.0.csv noise:0.113, imb:12.247,154,1886, Shape:(2040, 65)
('Spyder', 'BagNB')
237.22 [0.356, 0.544]
('Spyder', 'BagDT')
235.94 [0.234, 0.417]

14
groovy-1_5_7.csv noise:0.085, imb:8.463,80,677, Shape:(757, 65)
('Spyder', 'BagNB')
201.46 [0.207, 0.463]
('Spyder', 'BagDT')
201.1 [0.424, 0.46]

15
hbase-0.95.0.csv noise:0.234, imb:17.341,91,1578, Shape:(1669, 65)
('Spyder', 'BagNB')
221.18 [0.305, 0.472]
('Spyder', 'BagDT')
222.43 [0.163, 0.398]

16
camel-2.11.0.csv noise:0.024, imb:43.230,200,8646, Shape:(8846, 65)
('Spyder', 'BagNB')
699.87 [0.17, 0.23]
('Spyder', 'BagDT')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


698.76 [0.167, 0.246]

17
jruby-1.7.0.preview1.csv noise:0.099, imb:8.902,163,1451, Shape:(1614, 65)
('Spyder', 'BagNB')
214.71 [0.282, 0.402]
('Spyder', 'BagDT')
215.51 [0.281, 0.319]

18
hive-0.10.0.csv noise:0.126, imb:29.000,52,1508, Shape:(1560, 65)
('Spyder', 'BagNB')
218.2 [0.415, 0.453]
('Spyder', 'BagDT')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


217.7 [0.084, 0.354]

19
camel-2.10.0.csv noise:0.053, imb:24.447,311,7603, Shape:(7914, 65)
('Spyder', 'BagNB')
594.99 [0.239, 0.338]
('Spyder', 'BagDT')
594.65 [0.162, 0.176]

20
derby-10.2.1.6.csv noise:0.290, imb:9.906,180,1783, Shape:(1963, 65)
('Spyder', 'BagNB')
230.22 [0.437, 0.689]
('Spyder', 'BagDT')
228.87 [0.16, 0.631]

21
jruby-1.5.0.csv noise:0.218, imb:3.098,276,855, Shape:(1131, 65)
('Spyder', 'BagNB')
203.76 [0.329, 0.51]
('Spyder', 'BagDT')
205.55 [0.363, 0.419]

22
derby-10.3.1.4.csv noise:0.267, imb:13.051,157,2049, Shape:(2206, 65)
('Spyder', 'BagNB')
238.8 [0.402, 0.643]
('Spyder', 'BagDT')
242.28 [0.111, 0.573]

23
lucene-2.9.0.csv noise:0.226, imb:3.921,278,1090, Shape:(1368, 65)
('Spyder', 'BagNB')
216.1 [0.332, 0.455]
('Spyder', 'BagDT')
215.51 [0.307, 0.431]

24
hbase-0.94.0.csv noise:0.207, imb:14.348,69,990, Shape:(1059, 65)
('Spyder', 'BagNB')
209.4 [0.407, 0.558]
('Spyder', 'BagDT')
209.84 [0.269, 0.526]

25
camel-1.4.0.csv noise:0.281, imb:3.174,363,1152

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

251.93 [0.025, 0.16]

31
hive-0.9.0.csv noise:0.179, imb:25.717,53,1363, Shape:(1416, 65)
('Spyder', 'BagNB')
215.79 [0.442, 0.591]
('Spyder', 'BagDT')
215.86 [0.224, 0.534]

CPU times: user 1h 23min 36s, sys: 13min 4s, total: 1h 36min 40s
Wall time: 4h 41min 8s
